# First steps in PySpark 

In this notebook we will learn the fundamentals of functional programming, as well as the basic abstraction of a distributed object in Spark, the RDD. The notebook has been divided into two parts:

Part 1: map/reduce basics

Part 2: Work with RDD and Pair RDD abstractions 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

# Part 1: map/reduce basics

![Hadoop Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Hadoop_logo.svg/220px-Hadoop_logo.svg.png)
# **Apache Hadoop (MapReduce)**

It is an open source software framework written in Java for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with a fundamental assumption that hardware failures (of individual machines, or racks of machines) are common and thus should be automatically handled in software by the framework.

The core of Apache Hadoop consists of a storage part (Hadoop Distributed File System (HDFS)) and a processing part (MapReduce). Hadoop splits files into large blocks and distributes them amongst the nodes in the cluster. To process the data, Hadoop MapReduce transfers packaged code for nodes to process in parallel, based on the data each node needs to process. This approach takes advantage of data locality — nodes manipulating the data that they have on hand — to allow the data to be processed faster and more efficiently than it would be in a more conventional supercomputer architecture that relies on a parallel file system where computation and data are connected via high-speed networking.

![caption](http://d152j5tfobgaot.cloudfront.net/wp-content/uploads/2012/07/mapreduce.png)

Since data and computation are distributed, we should avoid the use of variables, i.e. mutable data. Thus, in contrast to impertaive programming, we shall use the functional approach (lambda calculus).

### The goal of the following excercises is to understand basic lambda calculus with python.

### (1a) Functional programming in Python

So, what is Functional Programming? From Wikipedia: 

« …a  programing paradigm that treats computation as the evaluation of mathematical functions and **avoids changing-state and mutable  data**.»

It´s based upon Lambda calculus, wich consist of:
 * Function definition (declaration of expressions)
 * Function application (evaluation of those expressions)
 * Recursion (iteration)

We have already used this in python!!! :)

Recall the typical "lambda x: x+1" we have been using as the first argument of map, reduce and filter methods:
 * **map** maps each value in the input collection to a different value. It´s just the classical mathematical funciton we are used to!
 * **reduce** takes two values from the input collection and returns a new value (of the same type) by appliying a commutative operation to them. 
 * **filter** filters the elements in the input collection according to a certain (boolean) criteria.
 

**Mapping**

`map` is a Higher Order Function (HOF) that takes a function f and a sequence and returns a new sequence formed by applying f to each element in the original sequence. 

![map](https://cosminpupaza.files.wordpress.com/2015/10/map.png?w=505)

Very often we write lambdas, anonymous functions, to use in defining simple transformations with `map`:


In [2]:
input_list = [1, 7, 98]
list(map(lambda n: n ** 2, input_list)) 

[1, 49, 9604]

In [4]:
list(map(lambda string: string + 's', ['coche', 'moto']))

['coches', 'motos']

We can also use named functions, of course. Note that we pass them as arguments to `map`, not execute them. You can think of them as tools we give to `map` for it to use.

In [6]:
def pluralize(word):
    return word + 's'

list(map(pluralize, ['coche', 'moto']))

['coches', 'motos']

Consider the following very common pattern: create empty list, append transformed elements one by one:

In [7]:
plurals = []

for element in ['coche', 'moto']:
    plurals.append(pluralize(element))
    
plurals

['coches', 'motos']

We can abstract over every possible list by defining a function that takes the list as an argument:

In [8]:
def pluralize_list(input_list):
    plurals = []

    for element in input_list:
        plurals.append(pluralize(element))

    return plurals

pluralize_list(['coche', 'moto'])

['coches', 'motos']

Consider this other piece of code: we are doing a very similar thing, but with a different function. A lot of the code is common between the two little programs.

Wouldn't it be convenient if we could abstract over every function, as we did over every list?

In [10]:
def initials(input_list):
    plurals = []

    for element in input_list:
        plurals.append(element[0])

    return plurals

initials(['coche', 'moto']) 

['c', 'm']

We can, taking the function to be applied as an argument. This is because Python treats functions as first-class objects, so they can be used as arguments to other functions.

In [11]:
def process_list(f, input_list):
    plurals = []

    for element in input_list:
        plurals.append(f(element))

    return plurals

process_list(pluralize, ['coche', 'moto']) 

['coches', 'motos']

In [12]:
process_list(lambda string: string[0], ['coche', 'moto'])

['c', 'm']

That function we just wrote, `process_list`, is exactly what `map` is

In [13]:
list(map(lambda string: string[0], ['coche', 'moto'])) 

['c', 'm']

In [16]:
absurd_animals = ['platypus', 'zebra', 'giraffe', 'naked mole rat']
sorted(absurd_animals)

['giraffe', 'naked mole rat', 'platypus', 'zebra']

In [17]:
sorted(absurd_animals, key=lambda word: word[-1])

['zebra', 'giraffe', 'platypus', 'naked mole rat']

In [18]:
sorted(absurd_animals, key=len)

['zebra', 'giraffe', 'platypus', 'naked mole rat']

**Filtering**

`filter` encodes another very common pattern: building a sequence from some of the elements of an input sequence, deciding whether to include each based on the result of evaluating a function, often called the `predicate`, on each of the elements.


![filter](https://cosminpupaza.files.wordpress.com/2015/11/filter.png?w=405)

In [19]:
input_list

[1, 7, 98]

In [20]:
list(filter(lambda n: n < 10, input_list)) 

[1, 7]

Again, we can use either lambdas or named functions.

In [21]:
def predicate(n):
    return n < 10

list(filter(predicate, input_list)) 

[1, 7]

#### Exercise

write a filter to extract those words that have more than 3 letters from `collection_of_strings`:



In [23]:
collection_of_strings = ['Holi', 'amigui', 'mii', 'vinti', 'a', 'mi', 'casita']

list(filter(lambda word: len(word) > 3, collection_of_strings))

['Holi', 'amigui', 'vinti', 'casita']

#### Exercise

Write a filter to extract prime numbers from the following list of integers:

```python
[12, 17, 19, 18, 23, 24]
```

You will need to write a function that determines whether a single number is a prime or not.

In [28]:
def isprime(number):
    
    for candidate in range(2, number):
        if number % candidate == 0:
            return False

    return True

isprime(57)

False

In [31]:
numbers = [12, 17, 19, 18, 23, 24]
list(filter(isprime, numbers)) 

[17, 19, 23]

`map` and `filter` are not  used explicitly very often in Python, in part because of their cumbersome syntax. 

Nevertheless, there is a well-known feature in Python that is in fact just a convenient way to write `map`s and `filter`s: **list comprehensions**:

In [34]:
numbers = [12, 17, 19, 18, 23, 24]

[ n for n in numbers if isprime(n) ] 

[17, 19, 23]

In [35]:
[ n ** 2 for n in numbers if isprime(n) ] 

[289, 361, 529]

The above list comprehension is exactly equivalent to:

In [37]:
list(map(lambda n : n ** 2, filter(isprime, numbers))) 

[289, 361, 529]

Which is a pipeline of two steps: first a `filter`, then a `map`

That's why no one uses filters and maps in Python

That's why no one uses filters and maps in Python

In [33]:
# Notice that a map of a Boolean function is not 
# at all the same as a filter with that function
[ isprime(n) for n in numbers ] 

[False, True, True, False, True, False]

**Reducing** 

`reduce` is the third basic foundation of functional programming. Reduce uses a function, often called the `combiner`, to transform a sequence of elements of type T (that is, any type) into a single T. The combiner must take 2 Ts and return only one T.


Recall it must be commutative! Think about the importance of this when parallelizing computations

![reduce](https://cosminpupaza.files.wordpress.com/2015/11/reduce.png?w=500)

In [39]:
from functools import reduce

reduce(lambda x, y: x + y, numbers)

113

In [40]:
# product

reduce(lambda x, y: x * y, numbers)

38511936

#### Exercise

Write a function_x such that, when applied with `reduce`, that reduce returns the highest number in a list. Don't use the built-in `max`!

In [41]:
max(numbers)

24

In [45]:
numbers

[12, 17, 19, 18, 23, 24]

In [44]:
def function_x(a, b):

    return a if a > b else b

reduce(function_x, numbers) 

24

## (1b) Exercise: Calculate the mean of a collection of real numbers using map/reduce
Recall:

$$\bar x = \frac{\sum_{i=1}^{N} x_i}{N} $$

It´s straightforward to do this with python built-in methods sum() and len(). However, how would you do that with map/reduce? We have already shown how to sum the elements of an array. Thus, you have to calculate the length of the array. For this:
 * Create another array of the same size, consisting of 1s.
 * Sum the elements of that array

#### First part

* Do a reduce to do the sum, and a different map-reduce to get the length

In [49]:
total = reduce(lambda x, y: x + y, numbers)
count = reduce(lambda x, y: x + y, map(lambda x: 1, numbers))

That was also a pipeline of two steps. It's equivalent to:

In [50]:
result_of_first_step = map(lambda x: 1, numbers)
reduce(lambda x, y: x + y, result_of_first_step)

6

#### Combine them in one pass

Think about what it would mean to finish one computation and only then do another in a distributed environment.

Combine both in one pass:

In [58]:
def f(n):
    return (n, 1)


def combiner(tuple_1, tuple_2):    
    running_total = tuple_1[0] + tuple_2[0]
    running_count = tuple_1[1] + tuple_2[1]

    return (running_total, running_count)


total, count = reduce(combiner, map(f, numbers))

total / count

18.833333333333332

## (1c) Exercise: Calculate the standard deviation of a collection of real numbers

We'll use the typical definition of the standard deviation:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1}$$

For this, use the *mean* and *count* variables from the previous excercise.

In [59]:
numbers

[12, 17, 19, 18, 23, 24]

In standard idiomatic Python we'd do it like this:

In [61]:
sum_square_differences = 0

for number in numbers:
    
    sum_square_differences += (number - (total / count)) ** 2
    
variance = sum_square_differences / (count - 1)
variance

18.96666666666667

With `map` and `reduce`:

Compare what we did earlier to transform this loop...

In [66]:
plurals = []

for element in ['coche', 'moto']:
    plurals += [element[0]]

plurals

['c', 'm']

...into this map:

In [65]:
list(map(lambda element: element[0], ['coche', 'moto']))

['c', 'm']

In [67]:
list(map(lambda number: (number - (total / count)) ** 2, numbers))

[46.69444444444443,
 3.3611111111111067,
 0.027777777777778172,
 0.6944444444444424,
 17.36111111111112,
 26.694444444444457]

In [71]:
reduce(lambda x, y: x + y, map(lambda n: (n - (total / count)) ** 2, numbers)) / (count - 1)

18.96666666666667

#### Side note

In Python 2, map and filter returned lists. In Python 3, they return generators, which are lazy collections. They are somewhat similar to files in that they can be depleted of elements after iterating through them.

In [77]:
%%time
import math

map(math.sqrt, range(100000000))

CPU times: user 21 µs, sys: 0 ns, total: 21 µs
Wall time: 28.6 µs


In [78]:
%%time
reduce(lambda x, y: x + y, map(math.sqrt, range(100000000)))

CPU times: user 13.2 s, sys: 0 ns, total: 13.2 s
Wall time: 13.2 s


666666661666.567

In [86]:
my_map = map(math.sqrt, range(5))
for number in my_map:
    print(number)

0.0
1.0
1.4142135623730951
1.7320508075688772
2.0


In [87]:
for number in my_map:
    print(number)

#### Calculate the standard deviation in one pass.

Again, think about what it would mean to finish one computation and only then do another in a distributed environment.


If we can, we should avoid broadcasting variables all over the cluster. In this case, the following alternative definition of standard deviation can come in handy:

$$\sigma_x^2 = \frac{\sum_{i=1}^{N} (x_i-\bar x)^2}{N-1} =
\frac{\sum_{i=1}^{N} (x_i^2+{\bar x}^2-2x_i\bar x)}{N-1} =
\frac{1}{N-1}\left(\sum_i x_i^2-N\bar x^2\right)$$



In [81]:
def f(n):
    return (n ** 2, n, 1)


def combiner(tuple_1, tuple_2):    
    running_sumsq = tuple_1[0] + tuple_2[0]
    running_total = tuple_1[1] + tuple_2[1]
    running_count = tuple_1[2] + tuple_2[2]

    return (running_sumsq, running_total, running_count)


sum_squares, total, count = reduce(combiner, map(f, numbers))

(sum_squares - count * (total / count) ** 2) / (count - 1)

18.966666666666697

## (1d) Twe 'word-count' problem: creating histograms
Given a set of keys in an input collection, calculate the frequency of each key. 

In order to understand better how map/reduce works, we will implement this simple calculation in several forms.

We are going to use as test case a list of elements chosen from a small set:

In [115]:
import random

random.choice(absurd_animals)

'naked mole rat'

In [75]:
random.seed(42)
my_absurd_zoo = [random.choice(absurd_animals) for _ in range(100)]

### (1d.1) Simple approach

 * Start with an empty dict
 * If a new key is not present in the dict, create it.
 * Otherwise, increase the frequency of the key by one.

In [76]:
headcount = {} 

for critter in my_absurd_zoo:
    if critter in headcount:
        headcount[critter] += 1
    else:
        headcount[critter] = 1
        
headcount

{'platypus': 23,
 'giraffe': 22,
 'zebra': 24,
 'star-nosed mole': 16,
 'naked mole rat': 15}

### (1d.2) Map/reduce

* Recall that *reduce* applies an operation to 2 elements of the same type, and returns another element of that type. Thus, the first thing to do is to map our collection to the type of the output. 
 
* The final result will be a dictionary of words in the vocabulary and counts. Therefore, we need to map each input word to this type.
 
* Then, we have to define a function, the combiner, that combines these dictionaries two at a time.

In [77]:
def combiner(left, right):
    
    for key in right:
        if key in left:
            left[key] += right[key]
        else:
            left[key] = right[key]
    
    return left


reduce(combiner, map(lambda bisho: {bisho: 1}, my_absurd_zoo))

assert(combiner({'giraffe' : 2}, {'dog' : 5, 'cat' : 4}) == {'giraffe' : 2, 'dog' : 5, 'cat' : 4})
assert(combiner({'giraffe' : 2, 'cow' : 8}, {'cow' : 0}) == {'giraffe' : 2, 'cow' : 8})

# Part 2: Spark. Work with RDD and Pair RDD abstractions 

![drawing](https://prateekvjoshi.files.wordpress.com/2015/10/1-main4.png)

# ** Apache Spark**

Apache Spark is an open source cluster computing framework originally developed in the AMPLab at University of California, Berkeley but was later donated to the Apache Software Foundation where it remains today. In contrast to Hadoop's two-stage disk-based MapReduce paradigm, Spark's multi-stage in-memory primitives provides performance up to 100 times faster for certain applications.

![](http://image.slidesharecdn.com/sparkandshark-120620130508-phpapp01/95/spark-and-shark-8-728.jpg?cb=1340197567)

By allowing user programs to load data into a cluster's memory and query it repeatedly, Spark is well-suited to machine learning algorithms.
![](http://spark.apache.org/images/logistic-regression.png)

Spark comes with a number of components that provide flexibility and generality.

<img src="http://spark.apache.org/images/spark-stack.png" alt="Drawing" style="width: 500px;"/>


## In this part, we keep on working on the word-count example, this time with spark. The basic abstraction of Spark is the Resilient Distributed Dataset (RDD):

#### «RDDs are fault-tolerant, parallel data structures that let users explicitly persist intermediate results in memory, control their partitioning to optimize data placement, and manipulate them using a rich set of operators.»

 * Read only, partitioned collection of records (immutable).
 * Stores the transformations used to build a dataset (its lineage), instead of the data itself. This property ensures fault-tolerance.
 * Users can control partitioning and persistence (caching).
 * RDDs are statically typed.
 * … and yes, everything is written in scala ;p. So you could use learning a bit of it!
 
<img src="http://eng.trueaccord.com/wp-content/uploads/2014/10/scala-logo.png" alt="Drawing" style="width: 200px;"/>

#### We will be trying to understand this abstraction with simple examples, using the [Python API](http://spark.apache.org/docs/latest/api/python/index.html)




### ** (2a) Create a base RDD: parallelize, actions and transformations **
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

We use sc.parallelize to convert a standard Python collection into an RDD.

In [74]:
import random

absurd_animals = ['platypus', 'zebra', 'giraffe', 'naked mole rat', 'star-nosed mole']
random.seed(42)
zoo = [random.choice(absurd_animals) for _ in range(100)]

In [5]:
zoo_rdd = sc.parallelize(zoo)

In [6]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [7]:
zoo_rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:175

In Spark, `map` is a method of the RDD type.

In [8]:
%%time
plurals = zoo_rdd.map(lambda animal: animal + 's')
plurals

CPU times: user 76 µs, sys: 12 µs, total: 88 µs
Wall time: 100 µs


**Nothing has actually happened!**

`parallellize` tells spark to distribute the data, but this is not actually done until we perform some action.

Possible actions include counting, collecting, reducing, taking, etc. Take a look at the [Spark programming guide](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions)


In [9]:
plurals.take(5) 

['platypuss', 'platypuss', 'giraffes', 'zebras', 'zebras']

In [11]:
all_plurals = plurals.collect()
len(all_plurals)

100

In [13]:
zoo_rdd.count()

100

#### Exercise

Calculate, from the rdd generated from absurd_animals, how many animals we have whose name does **not** start with an 's'

In [16]:
zoo_rdd.filter(lambda word: word[0] != 's').count()

84

A very common way to write these chained operations in Spark is to put each on a line. Due to Python's syntax, we need to escape the newline character with a backslash.

In [18]:
zoo_rdd.filter(lambda word: word[0] != 's')\
       .count()

84

### Actions and Transformations

There are two main types of operations in Spark:
[Actions](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#actions) and [Transformations](http://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#transformations).

**Transformations** produce an RDD. Some of the most important are `map`, `filter`, and `reduceByKey`. `sc.parallelize` and `sc.textFile` are not technically Transformations but you can think of them as such in one very important respect: they are *lazyly evaluated*. That is, when we perform a Transformation, we are only describing the operation to be performed at some point in the future. They attach a node to the execution graph.

**Actions** are implemented as methods on an RDD, and return an object of a type **that is not an RDD**. When we perform an Action, we give the order to execute the previously described transformations: we trigger the execution of the graph. Some of the most important are `reduce`, `collect`, `take`, `takeOrdered`, and `count`.

In [19]:
import math

In [24]:
%%time
rdd = sc.parallelize(range(100000000))

CPU times: user 3.97 ms, sys: 0 ns, total: 3.97 ms
Wall time: 5.41 ms


In [25]:
%%time
roots = rdd.map(math.sqrt)

CPU times: user 22 µs, sys: 3 µs, total: 25 µs
Wall time: 30.8 µs


In [26]:
%%time
roots.reduce(lambda x, y: x + y)

CPU times: user 7.6 ms, sys: 4.11 ms, total: 11.7 ms
Wall time: 4.49 s


666666661666.464

### **(2b) Persisting and the RDD lineage**

So far, we have seen that Spark RDDs are *lazyly evaluated*, i.e. nothing is actually done until an action is performed. In the RDD, the set of transformations to be applied are remembered: this is known as its *lineage*. It has the important consequence of making Spark RDDs *fault tolerant* automatically.

![](http://images.slideplayer.com/14/4499833/slides/slide_10.jpg) 

It might be interesting to store some intermediate results, though: perhaps because we want to apply several different transformations starting from that point, or because we are going to apply an iterative computation (as is customary in machine learning algorithms). For this, Spark has [several ways of persisting](http://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)

In [39]:
roots = sc.parallelize(range(100000000)).map(math.sqrt).cache()

In [40]:
% time roots.map(lambda a: -a).take(5)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 1.35 s


[-0.0, -1.0, -1.4142135623730951, -1.7320508075688772, -2.0]

In [41]:
% time roots.map(lambda a: -a).reduce(lambda x, y: x + y)

CPU times: user 3.47 ms, sys: 9.02 ms, total: 12.5 ms
Wall time: 8.73 s


-666666661666.464

In [42]:
% time roots.map(lambda a: -a).reduce(lambda x, y: x + y)

CPU times: user 10.8 ms, sys: 360 µs, total: 11.2 ms
Wall time: 8.79 s


-666666661666.464

In [46]:
roots.cache()
roots.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [47]:
from pyspark import StorageLevel

help(StorageLevel)

Help on class StorageLevel in module pyspark.storagelevel:

class StorageLevel(builtins.object)
 |  Flags for controlling the storage of an RDD. Each StorageLevel records whether to use memory,
 |  whether to drop the RDD to disk if it falls out of memory, whether to keep the data in memory
 |  in a JAVA-specific serialized format, and whether to replicate the RDD partitions on multiple
 |  nodes. Also contains static constants for some commonly used storage levels, MEMORY_ONLY.
 |  Since the data is always serialized on the Python side, all the constants use the serialized
 |  formats.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, useDisk, useMemory, useOffHeap, deserialized, replication=1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  

StorageLevel cannot be changed after it has been set. We need to `unpersist` first.

In [49]:
roots.unpersist()
roots.persist(StorageLevel.MEMORY_AND_DISK_2)

PythonRDD[20] at RDD at PythonRDD.scala:48

In [51]:
roots.persist(StorageLevel.DISK_ONLY_2)

Py4JJavaError: An error occurred while calling o181.persist.
: java.lang.UnsupportedOperationException: Cannot change storage level of an RDD after it was already assigned a level
	at org.apache.spark.rdd.RDD.persist(RDD.scala:170)
	at org.apache.spark.rdd.RDD.persist(RDD.scala:195)
	at org.apache.spark.api.java.JavaRDD.persist(JavaRDD.scala:47)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java

#### Intermission: Stack traces in Spark

Stack traces in pyspark are quite convoluted because we have many layers of processing. Look for the python stack trace contained within the Py4JJavaError message.

In [54]:
uppercase = roots.map(lambda root: root.upper())
uppercase.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 130, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1354, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-54-7bc949a98943>", line 1, in <lambda>
AttributeError: 'float' object has no attribute 'upper'

	at org

### **(2c) Partitioning **

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster.

To get the number of partitions of an RDD, just use `getNumPartitions()` on your RDD. You can change the partitions during RDD creation (with `parallelize(collection,numPartitions)` or `fromTextFile(file,numPartitions)`), or afterwards with methods like `repartition(), coalesce()`, etc.

In [58]:
zoo_rdd.take(5)

['platypus', 'platypus', 'giraffe', 'zebra', 'zebra']

In [59]:
zoo_rdd.getNumPartitions()

8

In [63]:
zoo_rdd = sc.parallelize(zoo, numSlices=10)
zoo_rdd.getNumPartitions()

10

We can see the partitions using [glom()](http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=glom#pyspark.RDD.glom): it returns an RDD created by coalescing all elements within each partition into a list.

In [65]:
glommed = zoo_rdd.glom()
glommed

PythonRDD[31] at RDD at PythonRDD.scala:48

In [66]:
glommed.collect()

[['platypus',
  'platypus',
  'giraffe',
  'zebra',
  'zebra',
  'zebra',
  'platypus',
  'star-nosed mole',
  'platypus',
  'star-nosed mole'],
 ['naked mole rat',
  'platypus',
  'platypus',
  'platypus',
  'zebra',
  'zebra',
  'star-nosed mole',
  'star-nosed mole',
  'platypus',
  'star-nosed mole'],
 ['zebra',
  'star-nosed mole',
  'naked mole rat',
  'zebra',
  'naked mole rat',
  'star-nosed mole',
  'giraffe',
  'platypus',
  'zebra',
  'naked mole rat'],
 ['giraffe',
  'giraffe',
  'zebra',
  'zebra',
  'giraffe',
  'platypus',
  'platypus',
  'naked mole rat',
  'platypus',
  'giraffe'],
 ['giraffe',
  'star-nosed mole',
  'giraffe',
  'platypus',
  'naked mole rat',
  'star-nosed mole',
  'platypus',
  'naked mole rat',
  'platypus',
  'star-nosed mole'],
 ['giraffe',
  'star-nosed mole',
  'giraffe',
  'star-nosed mole',
  'zebra',
  'platypus',
  'platypus',
  'zebra',
  'giraffe',
  'platypus'],
 ['zebra',
  'platypus',
  'naked mole rat',
  'giraffe',
  'naked mole rat

In [68]:
zebras = zoo_rdd.filter(lambda animal: animal=='zebra')
zebras.getNumPartitions()

10

In [69]:
zebras.glom().collect()

[['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 [],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra'],
 ['zebra', 'zebra'],
 ['zebra', 'zebra', 'zebra', 'zebra']]

Partitions are one of the most powerfull concepts in Spark: you can decide how to distribute your data so it can fit in memory, and more importantly, you can perform computations on each partition *before* speaking to other partitions. This can have an enormous impact on performance

### **(2c) Pair RDDs: *grouping* strategies in Spark**

The next step in writing our word counting program is to create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple (k, v) where k is the key and v is the value. In this example, we will create a pair consisting of ('<word>', 1) for each word element in the RDD, as we did in the map/reduce version of the histogram in Python, section (1d.2).

We can create the pair RDD using the map() transformation with a lambda() function to create a new RDD.

In [70]:
zoo_rdd.take(5)

['platypus', 'platypus', 'giraffe', 'zebra', 'zebra']

In [81]:
def dictcombiner(left, right):
    
    for key in right:
        if key in left:
            left[key] += right[key]
        else:
            left[key] = right[key]
    
    return left

In [82]:
zoo_rdd.map(lambda animal: {animal: 1}).reduce(dictcombiner) 

{'platypus': 23,
 'giraffe': 22,
 'zebra': 24,
 'star-nosed mole': 16,
 'naked mole rat': 15}

In [84]:
tuples = zoo_rdd.map(lambda animal: (animal, 1))
tuples.take(5)

[('platypus', 1), ('platypus', 1), ('giraffe', 1), ('zebra', 1), ('zebra', 1)]

### ** (2c.1) `groupByKey()` approach **
An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`. Next, sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [88]:
groups = tuples.groupByKey()
groups.collect()

[('zebra', <pyspark.resultiterable.ResultIterable at 0x7f2ecc23ff28>),
 ('giraffe', <pyspark.resultiterable.ResultIterable at 0x7f2ecc23fef0>),
 ('platypus', <pyspark.resultiterable.ResultIterable at 0x7f2ecc23fb38>),
 ('star-nosed mole',
  <pyspark.resultiterable.ResultIterable at 0x7f2ecc23fbe0>),
 ('naked mole rat', <pyspark.resultiterable.ResultIterable at 0x7f2ecc23fd68>)]

In [89]:
groups.count()

5

In [92]:
groups.map(lambda result_tuple: (result_tuple[0], len(result_tuple[1]))).collect() 

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

### ** (2c.2)  `reduceByKey` approach **
A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. 

The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

![](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/images/reduce_by.png)

In [93]:
help(tuples.reduceByKey)

Help on method reduceByKey in module pyspark.rdd:

reduceByKey(func, numPartitions=None, partitionFunc=<function portable_hash at 0x7f2ee43b2a60>) method of pyspark.rdd.PipelinedRDD instance
    Merge the values for each key using an associative and commutative reduce function.
    
    This will also perform the merging locally on each mapper before
    sending results to a reducer, similarly to a "combiner" in MapReduce.
    
    Output will be partitioned with C{numPartitions} partitions, or
    the default parallelism level if C{numPartitions} is not specified.
    Default partitioner is hash-partition.
    
    >>> from operator import add
    >>> rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
    >>> sorted(rdd.reduceByKey(add).collect())
    [('a', 2), ('b', 1)]



In [94]:
tuples.take(5)

[('platypus', 1), ('platypus', 1), ('giraffe', 1), ('zebra', 1), ('zebra', 1)]

In [96]:
counts = tuples.reduceByKey(lambda x, y: x + y)
counts

PythonRDD[75] at RDD at PythonRDD.scala:48

In [97]:
counts.collect() 

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

#### Summary: word count in Spark:

In [99]:
zoo_rdd.map(lambda animal: (animal, 1))\
       .reduceByKey(lambda x, y: x + y)\
       .collect()

[('zebra', 24),
 ('giraffe', 22),
 ('platypus', 23),
 ('star-nosed mole', 16),
 ('naked mole rat', 15)]

## (2d) Apply word count to a file

### ** (2d.1) Load a text file **
For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print(15 lines.)

In [101]:
!wget http://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt

--2018-10-06 10:12:08--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852404 (5,6M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   5,58M   797KB/s    in 9,8s    

2018-10-06 10:12:18 (586 KB/s) - ‘shakespeare.txt’ saved [5852404/5852404]



In [102]:
!head shakespeare.txt

In [105]:
shakespeare = sc.textFile('tocoto.txt')
shakespeare.take(5)

Py4JJavaError: An error occurred while calling o665.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/dani/repos/pyspark-course/tocoto.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:200)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.O

In [106]:
shakespeare = sc.textFile('shakespeare.txt')
shakespeare.take(5)

['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and']

### ** (2d.2) Capitalization and punctuation **
Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + All punctuation should be removed.
  + Any leading or trailing spaces on a line should be removed.
 
Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [107]:
lowercased = shakespeare.map(str.lower)
lowercased.take(5)

['',
 'project gutenberg’s the complete works of william shakespeare, by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and']

In [109]:
import re
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [113]:
def remove_punctuation(line):
    return re.sub('[^\w ]', '', line) 
    
remove_punctuation('sdflksajdf /.asdfad s.,fadfas d(ASFD) ')

'sdflksajdf asdfad sfadfas dASFD '

In [115]:
clean_lines = lowercased.map(remove_punctuation)
clean_lines.take(15) 

['',
 'project gutenbergs the complete works of william shakespeare by william',
 'shakespeare',
 '',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever  you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online at',
 'wwwgutenbergorg  if you are not located in the united states youll',
 'have to check the laws of the country where you are located before using',
 'this ebook',
 '',
 'see at the end of this file  content note added in 2017 ',
 '',
 '']

http://www.regular-expressions.info/quickstart.html

https://regex101.com/

### ** (2d.3) Words from lines **
Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + The first issue is that  that we need to split each line by its spaces.
  + The second issue is we need to filter out empty lines.
 
Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/3/library/stdtypes.html#str.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

Check out the Spark Programming Guide for an alternative transformation that helps us here.

In [117]:
remove_punctuation('sdflksajdf /.asdfad s.,fadfas d(ASFD) ').split() 

['sdflksajdf', 'asdfad', 'sfadfas', 'dASFD']

In [120]:
words = clean_lines.map(str.split)
words.take(3)

[[],
 ['project',
  'gutenbergs',
  'the',
  'complete',
  'works',
  'of',
  'william',
  'shakespeare',
  'by',
  'william'],
 ['shakespeare']]

In [122]:
words = clean_lines.flatMap(str.split)
words.take(10)

['project',
 'gutenbergs',
 'the',
 'complete',
 'works',
 'of',
 'william',
 'shakespeare',
 'by',
 'william']

### (2d.4) Count the words and show the top 15

We know the drill at this point, don't we? We map to a tuple then `reduceByKey`

We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pair tuples, we need a custom sort function that sorts using the value part of the pair rather than the key.

You'll notice that many of the words are common English words (know as stopwords).

Use our map reduce and and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [124]:
counts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
counts.take(15)

[('project', 107),
 ('of', 18815),
 ('shakespeare', 26),
 ('this', 7185),
 ('ebook', 14),
 ('is', 9709),
 ('use', 362),
 ('anyone', 9),
 ('anywhere', 8),
 ('in', 12032),
 ('united', 20),
 ('other', 723),
 ('world', 680),
 ('at', 2743),
 ('no', 4076)]

In [125]:
counts.takeOrdered(15)

[('1', 90),
 ('10', 3),
 ('100', 3),
 ('1000', 1),
 ('1000txt', 1),
 ('1000zip', 1),
 ('1004', 1),
 ('1009', 1),
 ('101', 1),
 ('1012', 1),
 ('1016', 1),
 ('102', 1),
 ('1020', 1),
 ('1024', 1),
 ('1028', 1)]

In [127]:
counts.takeOrdered(50, lambda element: -element[1])

[('the', 30002),
 ('and', 28358),
 ('i', 21867),
 ('to', 20816),
 ('of', 18815),
 ('a', 15992),
 ('you', 14437),
 ('my', 13191),
 ('in', 12032),
 ('that', 11781),
 ('is', 9709),
 ('not', 9067),
 ('with', 8518),
 ('me', 8270),
 ('for', 8187),
 ('it', 8174),
 ('his', 7572),
 ('be', 7365),
 ('this', 7185),
 ('your', 7079),
 ('he', 6804),
 ('but', 6771),
 ('have', 6268),
 ('as', 6179),
 ('thou', 5846),
 ('him', 5537),
 ('so', 5458),
 ('will', 5286),
 ('what', 4762),
 ('her', 4595),
 ('thy', 4357),
 ('all', 4216),
 ('by', 4093),
 ('no', 4076),
 ('do', 3912),
 ('shall', 3847),
 ('if', 3800),
 ('are', 3729),
 ('we', 3554),
 ('thee', 3381),
 ('on', 3316),
 ('our', 3296),
 ('lord', 3157),
 ('now', 3012),
 ('good', 2959),
 ('king', 2931),
 ('sir', 2900),
 ('from', 2899),
 ('o', 2765),
 ('at', 2743)]

# Practical

## ETL with airline coupon data

Load the data first: coupon data

In [129]:
data = sc.textFile('coupon150720.csv')
data.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

#### Exercise

Take fields 0, 2, 3, 4, and 6 from each line of coupons

In [134]:
def parse(line):

    fields = line.split(',')
    coupon = (fields[0], fields[2], fields[3], fields[4], float(fields[6]))
    
    return coupon # represented as a 5-element tuple

test_line = '79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'
parse(test_line)

('79062005698500', 'MAA', 'AUH', '9W', 56.79)

In [135]:
coupons = data.map(parse)
coupons.take(5)

[('79062005698500', 'MAA', 'AUH', '9W', 56.79),
 ('79062005698500', 'AUH', 'CDG', '9W', 84.34),
 ('79062005924069', 'CJB', 'MAA', '9W', 60.0),
 ('79065668570385', 'DEL', 'DXB', '9W', 160.63),
 ('79065668737021', 'AUH', 'IXE', '9W', 152.46)]

#### Exercise

Keep only the amount. Get average, max, min and std

In [138]:
sample = coupons.first()
sample[4]

56.79

In [140]:
amounts = coupons.map(lambda coupon: coupon[4])
amounts.take(5)

[56.79, 84.34, 60.0, 160.63, 152.46]

In [145]:
min_ = amounts.reduce(lambda x, y: x if x < y else y)
max_ = amounts.reduce(lambda x, y: x if x > y else y)

min_, max_

(0.0, 6355194.0)

In [157]:
tuple( a + b for a, b in zip([8,3,6], [1,2,3]) )

(9, 5, 9)

In [147]:
list(zip('abc', [1,2,3]))

[('a', 1), ('b', 2), ('c', 3)]

In [158]:
def combiner_tuples(tuple_1, tuple_2):
    return tuple( a + b for a, b in zip(tuple_1, tuple_2) )

In [160]:
sum_sq, total, count = amounts.map(lambda n: (n ** 2, n, 1))\
                              .reduce(combiner_tuples)
sum_sq, total, count                               

(122763999131054.75, 184831898.49994844, 1232662)

In [163]:
mean = total / count

std = ((sum_sq - count * mean ** 2) / (count - 1)) ** 0.5
std

9978.486133659524

#### All in one go!



In [164]:
tuples = amounts.map(lambda amount: (amount, amount, amount ** 2, amount, 1))
tuples.take(5)

[(56.79, 56.79, 3225.1041, 56.79, 1),
 (84.34, 84.34, 7113.235600000001, 84.34, 1),
 (60.0, 60.0, 3600.0, 60.0, 1),
 (160.63, 160.63, 25801.9969, 160.63, 1),
 (152.46, 152.46, 23244.051600000003, 152.46, 1)]

In [168]:
[max, min, lambda x, y: x + y, lambda x, y: x + y, lambda x, y: x + y,]

[<function max>,
 <function min>,
 <function __main__.<lambda>(x, y)>,
 <function __main__.<lambda>(x, y)>,
 <function __main__.<lambda>(x, y)>]

In [171]:
from operator import add

add(1, 2)

3

In [173]:
ops = [max, min, add, add, add] 
ops

[<function max>,
 <function min>,
 <function _operator.add>,
 <function _operator.add>,
 <function _operator.add>]

In [174]:
def combiner_tuples(tuple_1, tuple_2, ops):
    
    return tuple( op(a, b) for op, a, b in zip(ops, tuple_1, tuple_2) )

sample_tuple_1 = (56.79, 56.79, 3225.1041, 56.79, 1)
sample_tuple_2 = (84.34, 84.34, 7113.235600000001, 84.34, 1)

combiner_tuples(sample_tuple_1, sample_tuple_2, ops)

(84.34, 56.79, 10338.3397, 141.13, 2)

In [175]:
results = amounts.map(lambda amount: (amount, amount, amount ** 2, amount, 1))\
                 .reduce(lambda t1, t2: combiner_tuples(t1, t2, [max, min, add, add, add]))
results

(6355194.0, 0.0, 122763999131054.75, 184831898.49994844, 1232662)

#### Exercise

Get stats on ticket amount for all tickets with destination MAD

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

#### Part 1

We need to filter coupons with destination Madrid, and after that group on the origin. 

In [177]:
mad_coupons = coupons.filter(lambda coupon: coupon[2] == 'MAD')
mad_coupons.take(5)

[('79062005639642', 'BRU', 'MAD', 'UX', 21.02),
 ('79065668754871', 'BRU', 'MAD', 'SN', 27.66),
 ('79065668917696', 'CDG', 'MAD', 'AF', 46.97),
 ('79062006133090', 'CDG', 'MAD', 'AF', 3.38),
 ('79062006110497', 'CDG', 'MAD', 'AF', 26.02)]

At this point, we only need the origin, to group on, and the value, to sum

In [178]:
abbreviated = mad_coupons.map(lambda coupon: (coupon[1], coupon[4])) 
abbreviated.take(5)

[('BRU', 21.02), ('BRU', 27.66), ('CDG', 46.97), ('CDG', 3.38), ('CDG', 26.02)]

We are now ready to reduce all amounts per origin.

In [181]:
result = abbreviated.groupByKey().map(lambda tuple_: (tuple_[0], sum(tuple_[1])))
result.take(5)

[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.269999999953),
 ('VGO', 11697.699999999997),
 ('LCY', 10094.06)]

In [184]:
result = abbreviated.groupByKey().mapValues(lambda value: sum(value))
result.take(5)

[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.269999999953),
 ('VGO', 11697.699999999997),
 ('LCY', 10094.06)]

In [185]:
result = abbreviated.reduceByKey(lambda x, y: x + y)
result.take(5)

[('BRU', 21232.430000000008),
 ('TUN', 882.5100000000001),
 ('AMS', 29872.26999999995),
 ('VGO', 11697.699999999995),
 ('LCY', 10094.06)]

#### Part 2

This is very similar, with two differences: we need to group on the airline, and to calculate averages we need to keep track of both the total amount on the coupons and the number of coupons.

In [189]:
amounts = mad_coupons.map(lambda coupon: (coupon[3], coupon[4], 1))
amounts.take(5)

[('UX', 21.02, 1),
 ('SN', 27.66, 1),
 ('AF', 46.97, 1),
 ('AF', 3.38, 1),
 ('AF', 26.02, 1)]

In [192]:
amounts.reduceByKey(lambda _: '').take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 96.0 failed 1 times, most recent failure: Lost task 2.0 in stage 96.0 (TID 362, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 362, in func
    return f(iterator)
  File "/usr/local/spark/python

In [193]:
amounts = mad_coupons.map(lambda coupon: (coupon[3], (coupon[4], 1)))
amounts.take(5)

[('UX', (21.02, 1)),
 ('SN', (27.66, 1)),
 ('AF', (46.97, 1)),
 ('AF', (3.38, 1)),
 ('AF', (26.02, 1))]

Now we reduce tuples, summing each component of the tuple separately.

In [200]:
amounts.reduceByKey(lambda t1, t2: (t1[0]+ t2[0], t1[1] + t2[1])).take(5)

[('SN', (3092.6099999999997, 61)),
 ('IB', (928396.8400000058, 7526)),
 ('KL', (18007.62999999999, 323)),
 ('AZ', (3715.28, 54)),
 ('', (0.0, 168))]

In [203]:
amounts_by_airline = amounts.reduceByKey(lambda t1, t2: combiner_tuples(t1, t2, [add, add])).cache()

In [204]:
%time amounts_by_airline.take(5)

CPU times: user 7.85 ms, sys: 0 ns, total: 7.85 ms
Wall time: 1.25 s


[('SN', (3092.6099999999997, 61)),
 ('IB', (928396.8400000058, 7526)),
 ('KL', (18007.62999999999, 323)),
 ('AZ', (3715.28, 54)),
 ('', (0.0, 168))]

In [205]:
%time amounts_by_airline.take(5)

CPU times: user 5.92 ms, sys: 0 ns, total: 5.92 ms
Wall time: 17 ms


[('SN', (3092.6099999999997, 61)),
 ('IB', (928396.8400000058, 7526)),
 ('KL', (18007.62999999999, 323)),
 ('AZ', (3715.28, 54)),
 ('', (0.0, 168))]

This reduceByKey generates an RDD of the form (k, (v1, v2)). We can map it like this:

In [208]:
averages_by_airline = amounts_by_airline.map(lambda t: (t[0], t[1][0] / t[1][1]))
averages_by_airline.take(5)

[('SN', 50.69852459016393),
 ('IB', 123.35860217911318),
 ('KL', 55.751176470588206),
 ('AZ', 68.80148148148149),
 ('', 0.0)]

Or we can use the mapValues method to only process the (v1, v2) part, ignoring the key for processing without discarding it.

In [209]:
averages_by_airline = amounts_by_airline.mapValues(lambda t: t[0] / t[1])
averages_by_airline.take(5)

[('SN', 50.69852459016393),
 ('IB', 123.35860217911318),
 ('KL', 55.751176470588206),
 ('AZ', 68.80148148148149),
 ('', 0.0)]

And we're ready to take our result.

In [211]:
averages_by_airline.takeOrdered(10, lambda t: -t[1])

[('V0', 5418.098666666667),
 ('AC', 740.6200000000001),
 ('KE', 688.5261538461539),
 ('SV', 553.1742553191489),
 ('OB', 535.5044444444444),
 ('AR', 513.5304761904762),
 ('AV', 450.19509554140177),
 ('AM', 440.73421052631585),
 ('C2', 397.87),
 ('LA', 379.9537078651686)]

In [212]:
sc.textFile('coupon150720.csv')\
    .map(parse)\
    .filter(lambda coupon: coupon[2] == 'MAD')\
    .map(lambda coupon: (coupon[3], (coupon[4], 1)))\
    .reduceByKey(lambda t1, t2: combiner_tuples(t1, t2, [add, add]))\
    .mapValues(lambda t: t[0] / t[1])\
    .takeOrdered(10, lambda t: -t[1])

[('V0', 5418.098666666667),
 ('AC', 740.6200000000001),
 ('KE', 688.5261538461539),
 ('SV', 553.1742553191489),
 ('OB', 535.5044444444444),
 ('AR', 513.5304761904762),
 ('AV', 450.19509554140177),
 ('AM', 440.73421052631585),
 ('C2', 397.87),
 ('LA', 379.9537078651686)]

#### Stretch: 

Get the totals from first origin without using the coupon number

We'll use groupBy this time, because we need to look at every coupon in a ticket in order to identify first origin.

A ResultIterable is a lazy collection, so we can take it and iterate over it, or turn it into a list to materialize it. This is useful to get a test case to test the function we will be writing.

We are going to compare the set of origins to the set of destinations. The one that is in the first but not in the second should be the first origin.

Now we have have our values, so we are ready to get the totals and sort. Before we reduceByKey, we need to reshape our tuples so the first origin is the key.

### Further resources

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_Cheat_Sheet_Python.pdf